In [ ]:
# move into the folder
%cd drive/MyDrive/Dataset/pose_estimation/cerebella/tf-pose-estimation/

/content/drive/MyDrive/Dataset/pose_estimation/cerebella/tf-pose-estimation


In [ ]:
import sys
import time
import logging
import os
import pandas as pd
import random
import string
import tensorflow as tf
import numpy as np
from PIL import Image
import pickle
import matplotlib.pyplot as plt
import cv2 
from google.colab.patches import cv2_imshow
import math

In [ ]:
# read the csv in as an array
input = pd.read_csv('3d_updated_output.csv')
input

In [ ]:
def magnitude(vec1, vec2):
  diff = vec1 - vec2
  squared =  diff ** 2
  summed = sum(squared)
  mag = math.sqrt(summed)
  return(mag)

In [ ]:
# arguments: edges --> (s,e), (e,w), (s,w)
def angle_T1(se, ew, sw):
  # cosine of the shoulder vertex
  cosS = ((sw**2)+(se**2)-(ew**2))/(2*sw*se)
  beta = round(math.acos(cosS)*(180/(math.pi)),1)
  cosE = ((ew**2)+(se**2)-(sw**2))/(2*ew*se)
  alpha = round(math.acos(cosE)*(180/(math.pi)),1)
  cosW = ((sw**2)+(ew**2)-(se**2))/(2*sw*ew)
  gamma = round(math.acos(cosW)*(180/(math.pi)),1)

  result = np.array([beta,alpha,gamma])

  return result

In [ ]:
# arguments: edges --> (s,e), (h,e), (s,h)
def angle_T2(se, sh, he):
  # cosine of the shoulder vertex
  cosS = ((sh**2)+(se**2)-(he**2))/(2*sh*se)
  theta = round(math.acos(cosS)*(180/(math.pi)),1)
  cosE = ((se**2)+(he**2)-(sh**2))/(2*se*he)
  omega = round(math.acos(cosE)*(180/(math.pi)),1)
  cosH = ((sh**2)+(he**2)-(se**2))/(2*sh*he)
  phi = round(math.acos(cosH)*(180/(math.pi)))

  result = np.array([theta, omega, phi])

  return result

In [ ]:
# get the changes in degree and save them
delta_T1 = []
delta_T2 = []
delta_Z = []
# save the angles in each frame for now
angles_T1 = []
angles_T2 = []

# save the mean z values
zees = []

In [ ]:
# I need to figure out how to match the keypoints 
# with the current input .... DONE!

# 1 --> Hip
# 14 --> shoulder
# 15 --> elbow
# 16 --> wrist

In [ ]:
# get the data 
data = pd.read_csv('3d_updated_output.csv')

In [ ]:
# taking out the indexing column
del data['Unnamed: 0']

In [ ]:
# loop through the dataframe and select the values
# from col 1, 14, 15, 16
# hip
hip_x = data['0'].to_list()
hip_y = data['1'].to_list()
hip_z = data['2'].to_list()
# shoulder
shoulder_x = data['3'].to_list()
shoulder_y = data['4'].to_list()
shoulder_z = data['5'].to_list()
# elbow
elbow_x = data['6'].to_list()
elbow_y = data['7'].to_list()
elbow_z = data['8'].to_list()
# wrist
wrist_x = data['9'].to_list()
wrist_y = data['10'].to_list()
wrist_z = data['11'].to_list()

In [ ]:
type(elbow_y[0])

float

This next cell is a prep for the main loop.

In [ ]:
for i in range(0, len(hip)):
  x_h, space1, space2, y_h, z_h = hip[i].split(' ')
  x_h = float(x_h.split('[')[1])
  y_h = float(y_h)
  z_h = float(z_h.split(']')[0])
  hip_vec = [x_h, y_h]
  x_s, space1, space2, y_s, z_s = shoulder[i].split(' ')
  x_s = float(x_s.split('[')[1])
  y_s = float(y_s)
  z_s = float(z_s.split(']')[0])
  shoulder_vec = [x_s, y_s]
  x_e, space1, space2, y_e, z_e = elbow[i].split(' ')
  x_e = float(x_e.split('[')[1])
  y_e = float(y_e)
  z_e = float(z_e.split(']')[0])
  elbow_vec = [x_e, y_e]
  x_w, space1, space2, y_w, z_w = wrist[i].split(' ')
  x_w = float(x_w.split('[')[1])
  y_w = float(y_w)
  z_w = float(z_w.split(']')[0])
  wrist_vec = [x_w, y_w]
  z = [z_h, z_s, z_e, z_w] 

In [ ]:
# I want to run predictions on the new data

# I'm using 2 triangles to calsulate displacement
# Triangle 1: wrist(gamma), elbow(alpha), shoulder(beta)
# Triangle 2: shoulder(theta), hip(omega), elbow(phi)


# Ordering: shoulder, elbow, wrist, hip

# set global vars for prev vals:
# T1
orig_beta = 0
orig_alpha = 0
orig_gamma = 0
# T2
orig_theta = 0
orig_omega = 0
orig_phi = 0
# Z
orig_z = 0

for i in range(0, len(hip_x)):
  # hip
  x_h = hip_x[i] 
  y_h = hip_y[i] 
  hip_vec = np.array([x_h, y_h])
  z_h = hip_z[i] 
  # shoulder
  x_s = shoulder_x[i] 
  y_s = shoulder_y[i]
  shoulder_vec = np.array([x_s, y_s]) 
  z_s = shoulder_z[i]
  # elbow
  x_e = elbow_x[i] 
  y_e = elbow_y[i]
  elbow_vec = np.array([x_e, y_e]) 
  z_e = elbow_z[i]
  # wrist
  x_w = wrist_x[i] 
  y_w = wrist_y[i]
  wrist_vec = np.array([x_w, y_w]) 
  z_w = wrist_z[i] 


  # calculate the change in Z magnitudes
  z = [z_h, z_s, z_e, z_w] 
  z = np.array(z)
  z_avg = np.mean(z)
  zees.append(z_avg)
  change_in_z = z_avg - orig_z
  z_val = round(change_in_z,1)
  delta_Z.append(z_val)

  # calculate the magnitudes
  # T1: s,e,w
  se_mag = magnitude(shoulder_vec, elbow_vec)
  sw_mag = magnitude(shoulder_vec, wrist_vec)
  ew_mag = magnitude(elbow_vec, wrist_vec)

  # T2: s,e,h
  # I've aready calculated se in T1
  sh_mag = magnitude(shoulder_vec, hip_vec)
  he_mag = magnitude(hip_vec, elbow_vec)

  # calculate the angles
  results_T1 = angle_T1(se_mag,ew_mag,sw_mag)
  # print("results",  results_T1)
  angles_T1.append(results_T1.tolist())

  results_T2 = angle_T2(se_mag,sh_mag,he_mag)
  angles_T2.append(results_T2.tolist())

  # calculate the delta T1 & T2
  original_angles_T1 = np.array([orig_beta, orig_alpha, orig_gamma]) 
  T1_frame_delta = results_T1 - original_angles_T1
  # print("***>",results_T1)
  T1_delta_rounded = []
  # print("--->",results_T1[1],original_angles_T1[1])
  

  # for some reaon some values weren't rounding
  # properly so I'll take the long route out for now
  for unit in T1_frame_delta:
    val = round(unit,1)
    T1_delta_rounded.append(val)
  delta_T1.append(T1_delta_rounded)

  original_angles_T2 = np.array([orig_theta, orig_phi, orig_omega])
  T2_frame_delta = results_T2 - original_angles_T2
  T2_delta_rounded = []

  for unit in T2_frame_delta:
    val = round(unit,1)
    T2_delta_rounded.append(val)
  delta_T2.append(T2_delta_rounded)


  # replace the original angles with the current angles 
  orig_beta, orig_alpha, orig_gamma = angles_T1[-1]
  orig_theta, orig_phi, orig_omega = angles_T2[-1]
  orig_z = zees[-1]

In [ ]:
# Now I need to get the relevant deltas and make them a DF
deltas = []
for i in range(0, len(delta_T1)):
  holder = []
  holder.append(delta_T1[i][0])
  holder.append(delta_T2[i][1])
  holder.append(delta_T2[i][2])
  holder.append(delta_Z[i])

  deltas.append(holder)

In [ ]:
# make the deltas a DF
delta_df = pd.DataFrame(deltas)

In [ ]:
delta_df.to_csv("deltas.csv")